## Vector Search with IRIS SQL
This tutorial covers how to use IRIS as a vector database. 

For this tutorial, we will use a dataset of 2.2k online reviews of scotch (
dataset from https://www.kaggle.com/datasets/koki25ando/22000-scotch-whisky-reviews) . With our latest vector database functionality, we can leverage the latest embedding models to run semantic search on the online reviews of scotch whiskeys. In addition, we'll be able to apply filters on columns with structured data. For example, we will be able to search for whiskeys that are priced under $100, and are 'earthy, smooth, and easy to drink'. Let's find our perfect whiskey!

In [1]:
import os, pandas as pd
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, text

c:\Users\joels\anaconda3\envs\iris-vector-search\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [3]:
engine = create_engine(CONNECTION_STRING)

In [4]:
# dummy df to test, "image_path", "description"
df = pd.DataFrame({
    'image_path': ['image1.jpg', 'image2.jpg', 'image3.jpg'],
    'description': ['a cat', 'a dog', 'a car']
})

In [5]:
df.head()

,image_path,description
0,image1.jpg,a cat
1,image2.jpg,a dog
2,image3.jpg,a car


Now, InterSystems IRIS supports vectors as a datatype in tables! Here, we create a table with a few different columns. The last column, 'description_vector', will be used to store vectors that are generated by passing the 'description' of a review through an embedding model.

In [16]:
# drop table if exists
with engine.connect() as conn:
    with conn.begin():
        sql = f"""DROP TABLE IF EXISTS gallery_images"""
        result = conn.execute(text(sql))
        
        sql = f"""
                CREATE TABLE gallery_images (
                path VARCHAR(255),
                description VARCHAR(2000),
                description_vector VECTOR(DOUBLE, 1536)
        )
                """
        result = conn.execute(text(sql))

In [7]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [8]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df['description_vector'] = df['description'].apply(get_embedding)

df.head()

,image_path,description,description_vector
0,image1.jpg,a cat,"[-0.010084751062095165, -0.036302320659160614,..."
1,image2.jpg,a dog,"[-0.006658116355538368, -0.03515093773603439, ..."
2,image3.jpg,a car,"[0.004848200362175703, 0.005617125891149044, -..."


In [9]:
with engine.connect() as conn:
    with conn.begin():
        for index, row in df.iterrows():
            sql = text("""
                INSERT INTO gallery_images 
                (path, description, description_vector)
                VALUES (:path, :description, TO_VECTOR(:description_vector))
            """)
            conn.execute(sql, {
                'path': row['image_path'],
                'description': row['description'],
                'description_vector': str(row['description_vector'])
            })


Let's look for a scotch that costs less than $100, and has an earthy and creamy taste.

In [10]:
description_search = "likes fish and is very cute"
search_vector = get_embedding(description_search)

In [11]:
with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 3 * FROM gallery_images 
            ORDER BY VECTOR_DOT_PRODUCT(description_vector, TO_VECTOR(:search_vector)) DESC
        """)

        results = conn.execute(sql, {'search_vector': str(search_vector)}).fetchall()


In [12]:
print([t[1] for t in results])

['a cat', 'a dog', 'a car']


In [13]:
results_df = pd.DataFrame(results, columns=df.columns).iloc[:, :-1] # Remove vector
pd.set_option('display.max_colwidth', None)  # Easier to read description
results_df.head()

,image_path,description
0,image1.jpg,a cat
1,image2.jpg,a dog
2,image3.jpg,a car


### TESTING

In [1]:
from iris_db import IrisDB
import os
import pandas as pd

# openai key is in OPENAI_API_KEY.txt
OPENAI_API_KEY = open('OPENAI_API_KEY.txt').read().strip()
iris = IrisDB(OPENAI_API_KEY=OPENAI_API_KEY)

df = pd.DataFrame({
    'image_path': ['image1.jpg', 'image2.jpg', 'image3.jpg'],
    'description': ['a cat', 'a dog', 'a car']
})

iris.init_table()
iris.insert_df_to_table(df)

description_search = "likes fish and is very cute"
results = iris.description_search(description_search, top_n=3)
print([t[1] for t in results])

['a cat', 'a dog', 'a car']
